# Identification of shared peptides between M.musculus and C.alibcans proteomes

This notebook identifies tryptic peptides which are common between Mus musculus C57BL/6J and Candida albicans SC5314.

See the [README.md](https://github.com/bartongroup/Mouse-Candida-Peptide-Overlaps) file for full instructions on setting up and running these notebooks.

Running all the cells in the notebook will repeat the analysis. Outputs will be produced in a `data` subdirectory.

Note that much of the code is contained within the imported `PepOverlap.py` module

# Library Imports

In [ ]:
import os
import pandas as pd
import holoviews as hv
import matplotlib.pyplot as plt
import dill as pickle
from PepOverlap import *

from copy import deepcopy
from glob import glob
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from IPython.display import display, Markdown
from matplotlib_venn import venn2
from yaspin import yaspin

hv.extension('bokeh')

## Source data

### Mouse C57BL/J6

The MM10 mouse genome assembly is derived from C57BL/J6, so is ideal for this exercise. The proteins including all isoforms were downloaded from UniProt 2023_01, utilising the same URL as used to download the database for MS analysis.

### Candida albicans SC5314

Candida albicans SC5314 has been sequenced, and has the NCBI taxonomy identifier 237561. These proteins are similarly downloaded from UniProt 2023_01.

### Trypsin digests

Various tools are available for carrying out _in-silico_ trypsin digests. For the purposes of this exercise, a basic trypsin digest is adequeate. The EMBOSS 6.6.0.0 pepdigest tool can carry out trypsin digests, and by default does not cut proteins at unfavoured sites ('KR' followed by any of 'KRIFLP'). Trypsin digests are selected using the option '-menu 1', while it is also necessary to select whether to use monoisotopic weights or not for determination of peptide weights, but these aren't of relevance here. 

The following code cell will download the data, run pepdigest on the resulting fasta files and parse the results into Pandas dataframes, which are stored within a 'PepOverlap' object. Summary statistics of these are then produced and stored in the same object. The resulting output files will be retained in the `data` directory

In [ ]:
os.makedirs('data',exist_ok=True)

pep_data=PepOverlap()

beasts={
	'mouse':   "https://rest.uniprot.org/uniprotkb/stream?download=true&format=fasta&includeIsoform=true&query=(reviewed%3Atrue)%20AND%20(model_organism%3A10090)",
	'candida': "https://rest.uniprot.org/uniprotkb/stream?format=fasta&includeIsoform=true&query=%28%28taxonomy_id%3A237561%29%29"
}

for beast in beasts.keys():
	download(beasts[beast],beast)
	pepdigest(beast)

with yaspin(text="Processing results..."):
	pep_data.total_dfs['mouse'] = parse_pepdigest('mouse')
	pep_data.total_dfs['candida']= parse_pepdigest('candida')

	pep_data=summarise_proteome('mouse', pep_data)
	pep_data=summarise_proteome('candida', pep_data)


## Results

The total number of proteins and peptides represented, and also the proportion of proteins and peptides which are unique _within_ that organism are indicated in the table below the following cell. The unique proteins and peptides from each organism are then combined, and the unique numbers within this set also identified (labeled 'Combined unique peptides' in the table). This represents the set of peptides which are potentially identifiable by MS which can be attributed to a specific protein within a specific organism. The 'Unique proteins' and 'Unique peptides' values in this row indicate the proteins and peptides which are unique in the context of both organisms. 

In [ ]:
# Combined values from both genomes
merged_df=pd.concat(pep_data.unique_dfs)

merged_prots=merged_df['protein_id'].unique().tolist()
unique_merged_df=merged_df.drop_duplicates('peptide',keep=False)
unique_merged_prots=unique_merged_df['protein_id'].unique().tolist()

merged_data={
	'Category': 'Combined unique peptides',
	'Peptide count': len(merged_df.index),
	'Protein count': len(merged_prots),
	'Unique peptides': len(unique_merged_df.index),
	'Unique proteins': len(unique_merged_prots)
}
pep_data.summary_data.append(merged_data)

# Intersections between values for both genomes
duplicate_df=merged_df[merged_df.duplicated('peptide',keep=False)]
common_prots=duplicate_df['protein_id'].unique().tolist()

summary_columns=['Category','Protein count','Peptide count','Unique proteins', 'Unique peptides']
summary_df=pd.DataFrame(pep_data.summary_data,columns=summary_columns)

pep_data.duplicate_df=duplicate_df
with open('data/pep_data.pkl','wb') as fh:
	pickle.dump(pep_data,fh)

hv.Table(summary_df).opts(width=1000,title='Table 1: Summary of protein and peptide counts',fontsize={'title':12})

The intersections of various sets of peptides are shown in figure 1. The subset of *Candida albicans* and *Mus musculus* peptides which are unique within each organism are indicated in plots (a) and (b). These represent the number of peptides within each organism whose source can bot be categorically identified within an MS experiment involving that organism alone. The intersection between the full set of peptides from both organisms is indicated in (c), while (d) illustrates the number of peptides within the unique set of peptides from each organism. 

In [ ]:
plt.figure(figsize=(20, 15))
plt.subplot(221)
plt.title('(a) Candida albicans peptides',fontsize=20)

ca_pep=venn2(
		[set(pep_data.peplists_total['candida']),
		 set(pep_data.peplists_unique['candida'])],
		set_labels=(f"All peptides\n({len(pep_data.peplists_total['candida'])})", 
					f"Unique peptides\n({len(pep_data.peplists_unique['candida'])})")
	)
ca_pep=format_venn(ca_pep)
	
plt.subplot(222)
plt.title('(b) Mus musculus',fontsize=20)
mm_pep=venn2(
		[set(pep_data.peplists_total['mouse']),
		 set(pep_data.peplists_unique['mouse'] )],
		set_labels=(f"All peptides\n({len(pep_data.peplists_total['mouse'])})",
					f"Unique peptides\n({len(pep_data.peplists_unique['mouse'])})")
	)
mm_pep=format_venn(mm_pep)
	
plt.subplot(223)
plt.title('(c) Total peptides',fontsize=20)
total=venn2(
		[set(pep_data.peplists_total['candida']),
		 set(pep_data.peplists_total['mouse'])],
		set_labels=(f"Candida albicans\n({len(pep_data.peplists_total['candida'])})",
					f"Mus musculus\n({len(pep_data.peplists_total['mouse'])})")
	)
total=format_venn(total)

plt.subplot(224)
plt.title('(d) Unique peptides',fontsize=20)
unique=venn2(
		[set(pep_data.peplists_unique['candida']),
		 set(pep_data.peplists_unique['mouse'])],
		set_labels=(f"Candida albicans\n({len(pep_data.peplists_unique['candida'])})",
					f"Mus musculus\n({len(pep_data.peplists_unique['mouse'])})")
	)
unique=format_venn(unique)

# captures lists of protein IDs from each species
mouse_prot_ids=list(set(pep_data.total_dfs['mouse']['protein_id'].tolist()))
candida_prot_ids=list(set(pep_data.total_dfs['candida']['protein_id'].tolist()))

duplicate_df=merged_df[merged_df.duplicated('peptide',keep=False)]
common_prots=duplicate_df['protein_id'].unique().tolist()

plt.show()

*Figure 1 - Intersections of various peptide sets. (a) Unique peptides within total set of Candida albicans peptides (b) Unique peptides within the total set of Mus musculus peptides (c) Total peptides of Candida albicans and Mus musculus (d) Unique peptides from Candida albicans and Mus musculus.*

Note there is a discrepancy between the total number of peptides and the sum of the unique and non-unique peptides sets in each organism. This is a result of non-unique peptides which are represented more than twice, which is particularly common for very short peptides. 

## Shared Peptide Identification

Peptides which are shared between the two organisms are next isolated, and then proteins which share multiple peptides identified. This will inform on whether there are any proteins which are unlikely to be discriminated between due to containing insufficient unique peptides.

The summary table below is written to the `data` directory as a tab-delimited file: `shared_peptide_proportions.txt`. This lists all proteins which share more than 2 peptides across species, and the maximum number of 

In [ ]:
with yaspin(text='Analysing shared peptides...'):
	
	# Identify peptides shared between organisms i.e. peptides which are duplicated in the
	# dataframe of peptides merged from both species
	duplicate_df=merged_df[merged_df.duplicated('peptide',keep=False)]

	# Which proteins to these originate from?
	common_prots=duplicate_df['protein_id'].unique().tolist()

	# Which of these proteins contain multiple shared peptides?
	# i.e. where a protein ID is duplicated in the dataframe of duplicated peptides...
	multi_pep_common_prots=duplicate_df[duplicate_df.duplicated('protein_id',keep=False)]
	multi_pep_common_prot_ids=set(multi_pep_common_prots['protein_id'])

	shared_counts=[]

	for prot_id in multi_pep_common_prot_ids:
		common_peps=multi_pep_common_prots[multi_pep_common_prots['protein_id']==prot_id]
		total_peps=merged_df[merged_df['protein_id']==prot_id]
				
		shared_proportion=(float(len(common_peps.index))/float(len(total_peps.index))*100)
		
		if prot_id in mouse_prot_ids:
			species='Mouse'
		elif prot_id in candida_prot_ids:
			species='Candida'
		else:
			raise Exception('Protein ID not found')
			
		dat={
			'Protein ID':prot_id,
			'Species': species,
			'Shared peptides': len(common_peps.index),
			'Total peptides': len(total_peps.index),
			'Percentage shared': f'{shared_proportion:.2f}'
		}
		shared_counts.append(dat)

	columns=['Protein ID','Species', 'Shared peptides','Total peptides','Percentage shared']
	shared_count_df = pd.DataFrame(shared_counts,columns=columns)
	shared_count_df.to_csv('data/shared_peptide_proportions.txt',sep="\t",index=False)
	
hv.Table(shared_count_df,label='Proteins containing peptides shared between species').opts(width=800,show_title=True)

### Output Sequences

Protein sequences which are affected by shared peptides are written into the `data/mouse_intersection_proteins.fa` and `data/candida_intersection_proteins.fa` fasta files.

In [ ]:
# Save proteins with common peptides as fasta file
mouse_prots = SeqIO.to_dict(SeqIO.parse("data/mouse.fa", "fasta"))
candida_prots = SeqIO.to_dict(SeqIO.parse("data/candida.fa", "fasta"))

mouse_prot_records=[]
candida_prot_records=[]

for prot in common_prots:
	if prot in mouse_prots:
		record=mouse_prots[prot]
		mouse_prot_records.append(record)
	else:
		record=candida_prots[prot]
		candida_prot_records.append(record)

SeqIO.write(mouse_prot_records, "data/mouse_intersection_proteins.fa", "fasta")
SeqIO.write(candida_prot_records, "data/candida_intersection_proteins.fa", "fasta")

Additionally, a fasta file `data/shared_peptides.fa` containing the shared peptides which are not going to be discriminatory in MS analysis is created.

In [ ]:
shared_peptides=list(duplicate_df['peptide'].unique())

count=0
output_records=list()
for peptide in shared_peptides:
	count+=1
	record=SeqRecord(seq=Seq(peptide),id=f'{count:05d}',description='')
	output_records.append(record)

with open('data/shared_peptides.fa', 'w') as fh:
	SeqIO.write(output_records,fh,'fasta')


Finally, some summary tables of affected proteins...

In [ ]:
# Output summary tables of affected proteins
mouse_prot_info=get_prot_info(mouse_prot_records)
hv.Table(mouse_prot_info,label='Mouse proteins containing shared peptides').opts(width=1000)

In [ ]:
candida_prot_info=get_prot_info(candida_prot_records)
hv.Table(candida_prot_info,label='Candida proteins containing shared peptides').opts(width=1000)

## Summary

In [ ]:
display(Markdown(f'The overlap in peptides between Candida and mouse would result in certain peptides from {len(candida_prot_info)} Candida proteins and {len(mouse_prot_info)} proteins being unable to be assigned unambiguously to a specific protein in either organism.'))

This does not, however, mean that these proteins would not be able to be discriminated between by other peptides they contain, since in the vast majority of cases the proportion of unique peptides available within each organism is considerable. Only one short protein (containing only 3 peptides) protein was found to have >50% of it's peptides present in both organisms, so while individual peptides may be common between the organisms, there are plenty of peptides which are not shared which would allow proteins from the difference organisms to be identified

Considerably more peptides are non-unique *within* each organism than *between* the two organisms, and consequently would not allow discrimination between different proteins within each organism, than there are in the intersection between the organisms. Mouse, in particular, is notable for the proportion of non-unique peptides represented in the proteome. The originating organism and protein for the vast majority of peptides identified through MS proteomics experiments should therefore be able to be unambiguously identified.